In [1]:
from dataclasses import dataclass
from hashlib import sha256
from datetime import datetime
from flask import Flask, request
import requests
import rsa
import json

def getTime():
    now = datetime.now()
    time = now.strftime("%Y-%m-%d, %H:%M:%S")
    return time

def getHash(data):
    return sha256(data.encode()).hexdigest()

class blockchain:   
    def __init__(self):    
        self.chain = []
        self.NametoIpmap = {}
        self.NametoOwnermap = {}
        genesisBlock = block(0, 0x0, getTime(), 0x0)
        self.chain.append(genesisBlock)

    def getPreviousHash(self):
        return self.chain[-1].hash

    def getMapHash(self):
        mapHash = getHash(str(self.NametoIpmap) + str(self.NametoOwnermap))
        return mapHash

    def checkVaild(self):
        check = (self.chain[-1].mapHash == self.getMapHash())&(self.chain[-1].previousHash == self.chain[-2].hash)
        return check

    def addNewBlock(self):
        newBlock = block(len(self.chain), self.getPreviousHash(), getTime(), self.getMapHash())
        self.chain.append(newBlock)

    def addNewBinding(self, domainName, ip, owner):
        assert self.NametoIpmap.get(getHash(domainName), 'not exist') == 'not exist', 'Domain Name used'
        self.NametoIpmap[getHash(domainName)] = str(ip)
        self.NametoOwnermap[getHash(domainName)] = getHash(owner)
        self.addNewBlock()

    def changeBinding(self, domainName, Newip, owner):
        assert self.NametoOwnermap.get(getHash(domainName), 'not exist') != 'not exist', 'Domain Name not exist'
        assert self.NametoOwnermap.get(getHash(domainName), 'not exist') == getHash(owner), 'invaild user'
        self.NametoIpmap[getHash(domainName)] = str(Newip)
        self.addNewBlock()

    def queryBinding(self, domainName):
        assert self.checkVaild, 'invaild blockchain log'
        assert self.NametoIpmap.get(getHash(domainName), 'not exist') != 'not exist', 'Corresponding IP address not exist'
        return self.NametoIpmap[getHash(domainName)]

    def showBlock(self, index):
        selected_block = self.chain[index]
        print('{')
        print('index: ' + str(selected_block.index))
        print('previousHash: ' + str(selected_block.previousHash))
        print('timestamp: ' + str(selected_block.timestamp))
        print('mapHash: ' + str(selected_block.mapHash))
        print('} => Hash: ' + str(selected_block.hash))
        print('-'*80)

    def showAllBlock(self):
        for i in range(len(self.chain)):
            self.showBlock(i)

class block:
    def __init__(self, index, previousHash, timestamp, mapHash):    
        self.index = index
        self.previousHash = previousHash
        self.timestamp = timestamp
        self.mapHash = mapHash
        self.blockjson = {
                            'index': self.index,
                            'previousHash': self.previousHash,
                            'timestamp': self.timestamp,
                            'mapHash': self.mapHash,
                        }
        self.hash = sha256(json.dumps(self.blockjson).encode()).hexdigest()

class account:
    def __init__(self):
        (publickey, privatekey) = rsa.newkeys(512)
        self.privateKey = privatekey
        self.publicKey = publickey

    def encrypt(self, text):
        cipher = rsa.encrypt(bytes(text, encoding='utf-8'), self.publicKey)
        return cipher

    def decrypt(self, encrypt_text):
        plain = rsa.decrypt(encrypt_text, self.privateKey)
        return str(plain, encoding = "utf-8")

    def sign(self, hash):
        signature = rsa.sign(bytes(hash, encoding='utf-8'), self.privateKey, 'SHA-256')
        return signature

    def verify(self, signature, hash):
        method = rsa.verify(bytes(hash, encoding='utf-8'), signature, self.publicKey)
        return method == 'SHA-256'

In [2]:
chainA = blockchain()
userA = account()
userB = account()

chainA.queryBinding('baidu')

AssertionError: Corresponding IP address not exist

In [3]:
chainA.addNewBinding('baidu', '192.168.1.1', str(userA.publicKey))
chainA.showAllBlock()
chainA.checkVaild()
chainA.queryBinding('baidu')

{
index: 0
previousHash: 0
timestamp: 2023-03-06, 10:01:05
mapHash: 0
} => Hash: 36bbfc914f6e285efe5fb6455a046fda77b7953b4517f707592f1b4892d939ea
--------------------------------------------------------------------------------
{
index: 1
previousHash: 36bbfc914f6e285efe5fb6455a046fda77b7953b4517f707592f1b4892d939ea
timestamp: 2023-03-06, 10:01:10
mapHash: a232acc54f67159042e7b786dacc2a69e65945677cfdd8831973c7b7c7755839
} => Hash: fc780892b398c76e94be180e5b227bfdce191555566727bb7dc495492adcf038
--------------------------------------------------------------------------------


'192.168.1.1'

In [4]:
chainA.addNewBinding('baidu.com', '192.168.1.1', str(userA.publicKey))
chainA.queryBinding('baidu.com')

'192.168.1.1'

In [5]:
chainA.changeBinding('baidu.com', '192.168.1.2', str(userA.publicKey))
chainA.queryBinding('baidu.com')

'192.168.1.2'

In [6]:
chainA.changeBinding('baidu.com', '192.168.1.2', str(userB.publicKey))

AssertionError: invaild user

In [7]:
chainA.changeBinding('www.baidu.com', '192.168.1.2', str(userB.publicKey))

AssertionError: Domain Name not exist

In [8]:
chainA.showAllBlock()

chainA.NametoIpmap

{
index: 0
previousHash: 0
timestamp: 2023-03-06, 10:01:05
mapHash: 0
} => Hash: 36bbfc914f6e285efe5fb6455a046fda77b7953b4517f707592f1b4892d939ea
--------------------------------------------------------------------------------
{
index: 1
previousHash: 36bbfc914f6e285efe5fb6455a046fda77b7953b4517f707592f1b4892d939ea
timestamp: 2023-03-06, 10:01:10
mapHash: a232acc54f67159042e7b786dacc2a69e65945677cfdd8831973c7b7c7755839
} => Hash: fc780892b398c76e94be180e5b227bfdce191555566727bb7dc495492adcf038
--------------------------------------------------------------------------------
{
index: 2
previousHash: fc780892b398c76e94be180e5b227bfdce191555566727bb7dc495492adcf038
timestamp: 2023-03-06, 10:01:13
mapHash: 0f70df8838853caa22bda937c29e0338d30ac8916cfede27d718b66438525032
} => Hash: ddebc8fc2e02cc90b9eec48c5e936d16d373eb8c6336f4b013347033b8961edc
--------------------------------------------------------------------------------
{
index: 3
previousHash: ddebc8fc2e02cc90b9eec48c5e936d16d373eb8c63

{'a89151ba4b5ac0f22e96b71b963db927791d3808f5175f06ae4a60de5891bf0f': '192.168.1.1',
 '79a12352634d1a1588b639d3ca5140b71bc83e155b491fb6a4940c1c8fbe9fdb': '192.168.1.2'}

In [9]:
chainA.NametoOwnermap

{'a89151ba4b5ac0f22e96b71b963db927791d3808f5175f06ae4a60de5891bf0f': '48ac2e11579e99ceb25d89563fa5cbca68278355fa9771fdae0d993b785126fc',
 '79a12352634d1a1588b639d3ca5140b71bc83e155b491fb6a4940c1c8fbe9fdb': '48ac2e11579e99ceb25d89563fa5cbca68278355fa9771fdae0d993b785126fc'}